In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
import plotly.express as px


from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans

%matplotlib inline

In [ ]:
df = pd.read_csv('sales_data_sample.csv', encoding = 'unicode_escape', parse_dates=['ORDERDATE'])
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df_drop  = ['ADDRESSLINE1', 'ADDRESSLINE2', 'POSTALCODE', 'CITY', 'TERRITORY', 'PHONE', 'STATE', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'CUSTOMERNAME', 'ORDERNUMBER']
df = df.drop(df_drop, axis=1)
df.head(3)

#### Drop georaphic features and names, phone

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
def barplot_visualization(x):
  fig = plt.Figure(figsize = (12, 6))
  fig = px.bar(x = df[x].value_counts().index, y = df[x].value_counts(), color = df[x].value_counts().index, height = 600)
  fig.show();

In [ ]:
barplot_visualization('COUNTRY')

In [ ]:
barplot_visualization('STATUS');

#### Drop unbalanced feature

In [ ]:
df.drop(columns=['STATUS'], axis=1, inplace=True)

In [ ]:
print('Columns resume: ', df.shape[1])

In [ ]:
barplot_visualization('PRODUCTLINE')

In [ ]:
barplot_visualization('DEALSIZE')

### Prepare data

In [ ]:
def dummies(x):
  dummy = pd.get_dummies(df[x])
  df.drop(columns=x, inplace=True)
  return pd.concat([df, dummy], axis = 1)

In [ ]:
df =  dummies('COUNTRY')

In [ ]:
df =  dummies('PRODUCTLINE')

In [ ]:
df =  dummies('DEALSIZE')

In [ ]:
df.head()

In [ ]:
y = pd.Categorical(df['PRODUCTCODE'])
y

In [ ]:
df['PRODUCTCODE'] = pd.Categorical(df['PRODUCTCODE']).codes

In [ ]:
df.head()

In [ ]:
df_group = df.groupby(by='ORDERDATE').sum()
fig = px.line(x = df_group.index, y = df_group.SALES, title='sales_peak')
fig.show();

In [ ]:
df.drop('ORDERDATE', axis=1, inplace=True)

#### drop 'ORDERDATE', 'QTR_ID' because we have 'MONTH' etc.

In [ ]:
df.drop('QTR_ID', axis=1, inplace=True)

In [ ]:
df.shape

## Use K-MEANS algorithm

In [ ]:
scaler =  StandardScaler()
df_scaled = scaler.fit_transform(df)

In [ ]:
scores = []
range_values = range(1, 15)
for i in range_values:
  kmeans = KMeans(n_clusters = i)
  kmeans.fit(df_scaled)
  scores.append(kmeans.inertia_)

In [ ]:
plt.plot(scores, 'bx-')
plt.title('Finding right number of clusters')
plt.xlabel('Clusters')
plt.ylabel('scores')
plt.show();

#### The elbow method

In [ ]:
kmeans = KMeans(4)
kmeans.fit(df_scaled)

In [ ]:
labels = kmeans.labels_
labels

In [ ]:
kmeans.cluster_centers_.shape

In [ ]:
cluster_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns= [df.columns])
cluster_centers

#### Invert the data

In [ ]:
cluster_centers = scaler.inverse_transform(cluster_centers)
cluster_centers = pd.DataFrame(data=cluster_centers, columns=[df.columns])
cluster_centers

In [ ]:
sales_of_cluster = pd.concat([df, pd.DataFrame({'cluster': labels})], axis=1)
sales_of_cluster.head()